In [6]:
import ee
import geemap

In [3]:
ee.Authenticate()

Enter verification code: 4/1AX4XfWiEGHzpaGGeVPcosVAWVcxBjmDtd_x-c69L3FnkErOc8p9XALSVrNg

Successfully saved authorization token.


In [4]:

ee.Initialize()

In [2]:
js_snippet = """

//adding points


var points = ee.FeatureCollection("projects/fire-ashoka/assets/145044_20100428_500points");

Map.addLayer(points, {color:'red'}, '500points')

var image = ee.Image('LANDSAT/LT05/C01/T1_TOA/LT05_145044_20100428')
            .select(['B1', 'B2', 'B3', 'B4', 'B5', 'B7']);

Map.centerObject(image)


var elevation_raster = ee.Image('USGS/SRTMGL1_003');

// Calculate Slope and Elevation

var elev = elevation_raster.select('elevation').rename('elev');
var slope = ee.Terrain.slope(elevation_raster.select('elevation')).rename('slope');

var image = image.addBands(elev).addBands(slope)


// hillshadow calc

//ee.Terrain.hillShadow(image, azimuth, zenith, neighborhoodSize, hysteresis)

var getShadow = function getShadow(image){
  
  var elevation = image.get('SUN_ELEVATION')
  var zenith = ee.Number(90).subtract(ee.Number(elevation))
  
  var shadow = ee.Terrain.hillShadow({
    image: elevation_raster,
    azimuth: image.get('SUN_AZIMUTH'),
    zenith: ee.Number(zenith),
    neighborhoodSize: 200,
    hysteresis: true
  }).rename('shadow');

  return image.addBands(shadow)
}

var image = getShadow(image)




// var getShadow = function getShadow(image){

//   var azimuth = image.get('SUN_AZIMUTH')
  
//   var elevation = image.get('SUN_ELEVATION')

//   var zenith = ee.Number(90).subtract(ee.Number(elevation))
//   var shadow = ee.Terrain.hillShadow(image, azimuth, zenith)
  
//   return image.addBands(shadow)
// }

// var image  = getShadow(image)

/// indices creation

var addIndices = function indices(image){


  var ndvi = image.normalizedDifference(['B4', 'B3']).rename('ndvi');
  
  var ndmi = image.normalizedDifference(['B4', 'B5']).rename('ndmi');
  
  var nbr = image.normalizedDifference(['B4', 'B7']).rename('nbr');
  
  var ndwi = image.normalizedDifference(['B2', 'B4']).rename('ndwi');
  
  var bai = image.expression(
    '1/((0.1-red)**2+ (0.06-nir)**2)',{
      'nir': image.select('B4'),
      'red': image.select('B3')
      
    }).rename('bai');
    
  return image.addBands(ndvi).addBands(ndmi).addBands(nbr).addBands(ndwi).addBands(bai)

};

// Expression using image band indices.
// var bandIndexExp = '(b(4) - b(3)) / (b(4) + b(3))';
// var bandIndexImg = img.expression(bandIndexExp).rename('NDVI');

var mirbiExp = '(10 * b(7) - ((9.8 * b(5)+ 2)'

var mirbi = image.expression(mirbiExp).rename('mirbi')

print('mirbi')



var image = addIndices(image);



// Normalize the image 

// need a boundar for this

//Map.addLayer(bai, {min :0, max : 100}, 'bai')
          
var product_ID = image.get('LANDSAT_PRODUCT_ID');

print('Product ID : ', product_ID)

var properties = image.propertyNames();
print('Metadata properties:', properties);  // ee.List of metadata properties



// // Reduce the image to get a one-band maximum value image.
// var maxValue = image.reduce(ee.Reducer.max());


var rgbVis = {
  min:0,
  max :0.4,
  bands : ['B5', 'B4', 'B3'],
  gamma : 1.2
}

Map.addLayer(image, rgbVis, 'original')

// splitting the data

var points = points.randomColumn()

//30%, 70%

var trainingPoints = points.filter(ee.Filter.lt('random', .7));
var validationPoints = points.filter(ee.Filter.gte('random', .7))

//overlaying the points on the image to get the training data

var training = image.sampleRegions({
  collection : trainingPoints, 
  properties : ['class'],
  scale: 30
});
print(image.bandNames())

print(training)

// so it's not taking the no data?


//training classifier

var classifier = ee.Classifier.smileRandomForest(500).train({
  features: training,
  classProperty: 'class',
  inputProperties : image.bandNames()
});

//classifying the image

var classified = image.classify(classifier);

var palette = ['darkgreen', "orange", 'yellow',  
          "darkorange","brown","lightblue",
          '#E69F00','lightgreen', 'grey', 'green' ]

Map.addLayer(classified, {min:0, max : 10, palette :palette}, 'classified' )



// accuracy assessment

var test = classified.sampleRegions({
  collection : validationPoints,
  properties : ['class'],
  scale: 30
})

print(test)


var testConfusionMatrix = test.errorMatrix('class', 'classification')

print('Confusion Matrix ', testConfusionMatrix);

print('Test Accuracy', testConfusionMatrix.accuracy())


// Create a Feature with null geometry and the value we want to export.
// Use .array() to convert Confusion Matrix to an Array so it can be
// exported in a CSV file

var fc = ee.FeatureCollection([
  ee.Feature(null, {
    'accuracy': testConfusionMatrix.accuracy(),
    'matrix': testConfusionMatrix.array()
  })
  ])
print(fc)  
Export.table.toDrive({
  collection: fc,
  description: 'Accuracy_Export',
  folder: 'ee',
  fileNamePrefix: 'accuracy',
  fileFormat: 'CSV'
})


//// Calculate variable importance
var importance = ee.Dictionary(classifier.explain().get('importance'))

// Calculate relative importance
var sum = importance.values().reduce(ee.Reducer.sum())

var relativeImportance = importance.map(function(key, val) {
  return (ee.Number(val).multiply(100)).divide(sum)
  })
print(relativeImportance)

// Create a FeatureCollection so we can chart it
var importanceFc = ee.FeatureCollection([
  ee.Feature(null, relativeImportance)
])

var chart = ui.Chart.feature.byProperty({
  features: importanceFc
}).setOptions({
      title: 'Feature Importance',
      vAxis: {title: 'Importance'},
      hAxis: {title: 'Feature'}
  })
print(chart)

// Tune the numberOfTrees parameter.\


var test = image.sampleRegions({
  collection : validationPoints,
  properties : ['class'],
  scale: 30
})

print(test)
var numTreesList = ee.List.sequence(50, 1000, 50);

var accuracies = numTreesList.map(function(numTrees) {
  var classifier = ee.Classifier.smileRandomForest(numTrees)
      .train({
        features: training,
        classProperty: 'class',
        inputProperties: image.bandNames()
      });

  // Here we are classifying a table instead of an image
  // Classifiers work on both images and tables
  return test
    .classify(classifier)
    .errorMatrix('class', 'classification')
    .accuracy();
});
print(accuracies)

var chart = ui.Chart.array.values({
  array: ee.Array(accuracies),
  axis: 0,
  xLabels: numTreesList
  }).setOptions({
      title: 'Hyperparameter Tuning for the numberOfTrees Parameters',
      vAxis: {title: 'Validation Accuracy'},
      hAxis: {title: 'Number of Tress', gridlines: {count: 15}}
  });
print(chart)

// Tuning Multiple Parameters
// numTrees and bagFraction 
var numTreesList = ee.List.sequence(50,1000, 50);
var bagFractionList = ee.List.sequence(0.1, 0.9, 0.1);

var accuracies = numTreesList.map(function(numTrees) {
  return bagFractionList.map(function(bagFraction) {
     var classifier = ee.Classifier.smileRandomForest({
       numberOfTrees: numTrees,
       bagFraction: bagFraction
     })
      .train({
        features: training,
        classProperty: 'class',
        inputProperties: image.bandNames()
      });

    // Here we are classifying a table instead of an image
    // Classifiers work on both images and tables
    var accuracy = test
      .classify(classifier)
      .errorMatrix('class', 'classification')
      .accuracy();
    return ee.Feature(null, {'accuracy': accuracy,
      'numberOfTrees': numTrees,
      'bagFraction': bagFraction})
  })
}).flatten()
var resultFc = ee.FeatureCollection(accuracies)

// Export the result as CSV
Export.table.toDrive({
  collection: resultFc,
  description: 'Multiple_Parameter_Tuning_Results',
  folder: 'ee',
  fileNamePrefix: 'numtrees_bagfraction',
  fileFormat: 'CSV'}) 
  




 
"""

In [7]:
js_snippet = """
// FeatureCollection of power plants in Belgium.
var fc = ee.FeatureCollection('WRI/GPPD/power_plants')
            .filter('country_lg == "Belgium"');

// A list of feature properties to combine into an array
// (power generation by year).
var properties = ['gwh_2013', 'gwh_2014', 'gwh_2015', 'gwh_2016'];

// Add array of power-generation-by-year property to features.
fc = fc.makeArray(properties, 'gwh_by_year');

print('FeatureCollection with array of selected properties added', fc);
print('See example of new "gwh_by_year" property', fc.first().toDictionary());
"""

In [8]:
geemap.js_snippet_to_py(
    js_snippet, add_new_cell=True, import_ee=True, import_geemap=True, show_map=True
)

In [ ]:
import ee
import geemap

Map = geemap.Map()
import geemap

Map = geemap.Map()

# FeatureCollection of power plants in Belgium.
fc = ee.FeatureCollection('WRI/GPPD/power_plants') \
            .filter('country_lg == "Belgium"')

# A list of feature properties to combine into an array
# (power generation by year).
properties = ['gwh_2013', 'gwh_2014', 'gwh_2015', 'gwh_2016']

# Add array of power-generation-by-year property to features.
fc = fc.makeArray(properties, 'gwh_by_year')

print('FeatureCollection with array of selected properties added', fc)
print('See example of new "gwh_by_year" property', fc.first().toDictionary())
MapMap


In [ ]:
import ee
import geemap

Map = geemap.Map()
import geemap

Map = geemap.Map()

# FeatureCollection of power plants in Belgium.
fc = ee.FeatureCollection('WRI/GPPD/power_plants') \
            .filter('country_lg == "Belgium"')

# A list of feature properties to combine into an array
# (power generation by year).
properties = ['gwh_2013', 'gwh_2014', 'gwh_2015', 'gwh_2016']

# Add array of power-generation-by-year property to features.
fc = fc.makeArray(properties, 'gwh_by_year')

print('FeatureCollection with array of selected properties added', fc)
print('See example of new "gwh_by_year" property', fc.first().toDictionary())
MapMap


In [ ]:
import ee
import geemap

Map = geemap.Map()
import geemap

Map = geemap.Map()

#adding points

points = ee.FeatureCollection("projects/fire-ashoka/assets/145044_20100428_500points")

Map.addLayer(points, {'color':'red'}, '500points')

image = ee.Image('LANDSAT/LT05/C01/T1_TOA/LT05_145044_20100428') \
            .select(['B1', 'B2', 'B3', 'B4', 'B5', 'B7'])

Map.centerObject(image)

elevation_raster = ee.Image('USGS/SRTMGL1_003')

# Calculate Slope and Elevation

elev = elevation_raster.select('elevation').rename('elev')
slope = ee.Terrain.slope(elevation_raster.select('elevation')).rename('slope')

image = image.addBands(elev).addBands(slope)

# hillshadow calc

#ee.Terrain.hillShadow(image, azimuth, zenith, neighborhoodSize, hysteresis)

def getShadow getShadow(image):

  elevation = image.get('SUN_ELEVATION')
  zenith = ee.Number(90).subtract(ee.Number(elevation))

  shadow = ee.Terrain.hillShadow({
    'image': elevation_raster,
    'azimuth': image.get('SUN_AZIMUTH'),
    'zenith': ee.Number(zenith),
    'neighborhoodSize': 200,
    'hysteresis': True
  }).rename('shadow')

  return image.addBands(shadow)

image = getShadow(image)

# def getShadow getShadow(image):

#   azimuth = image.get('SUN_AZIMUTH')

#   elevation = image.get('SUN_ELEVATION')

#   zenith = ee.Number(90).subtract(ee.Number(elevation))
#   shadow = ee.Terrain.hillShadow(image, azimuth, zenith)

#   return image.addBands(shadow)
#

# image  = getShadow(image)

#/ indices creation

def addIndices indices(image):

  ndvi = image.normalizedDifference(['B4', 'B3']).rename('ndvi')

  ndmi = image.normalizedDifference(['B4', 'B5']).rename('ndmi')

  nbr = image.normalizedDifference(['B4', 'B7']).rename('nbr')

  ndwi = image.normalizedDifference(['B2', 'B4']).rename('ndwi')

  bai = image.expression(
    '1/((0.1-red)**2+ (0.06-nir)**2)',{
      'nir': image.select('B4'),
      'red': image.select('B3')

    }).rename('bai')

  return image.addBands(ndvi).addBands(ndmi).addBands(nbr).addBands(ndwi).addBands(bai)

# Expression using image band indices.
# bandIndexExp = '(b(4) - b(3)) / (b(4) + b(3))'
# bandIndexImg = img.expression(bandIndexExp).rename('NDVI')

mirbiExp = '(10 * b(7) - ((9.8 * b(5)+ 2)'

mirbi = image.expression(mirbiExp).rename('mirbi')

print('mirbi')

image = addIndices(image)

# Normalize the image

# need a boundar for this

#Map.addLayer(bai, {min :0, max : 100}, 'bai')

product_ID = image.get('LANDSAT_PRODUCT_ID')

print('Product ID : ', product_ID)

properties = image.propertyNames()
print('Metadata properties:', properties);  # ee.List of metadata properties

# # Reduce the image to get a one-band maximum value image.
# maxValue = image.reduce(ee.Reducer.max())

rgbVis = {
  'min':0,
  'max' :0.4,
  'bands' : ['B5', 'B4', 'B3'],
  'gamma' : 1.2
}

Map.addLayer(image, rgbVis, 'original')

# splitting the data

points = points.randomColumn()

#30%, 70%

trainingPoints = points.filter(ee.Filter.lt('random', .7))
validationPoints = points.filter(ee.Filter.gte('random', .7))

#overlaying the points on the image to get the training data

training = image.sampleRegions({
  'collection' : trainingPoints,
  'properties' : ['class'],
  'scale': 30
})
print(image.bandNames())

print(training)

# so it's not taking the no data?

#training classifier

classifier = ee.Classifier.smileRandomForest(500).train({
  'features': training,
  'classProperty': 'class',
  'inputProperties' : image.bandNames()
})

#classifying the image

classified = image.classify(classifier)

palette = ['darkgreen', "orange", 'yellow',
          "darkorange","brown","lightblue",
          '#E69F00','lightgreen', 'grey', 'green' ]

Map.addLayer(classified, {'min':0, 'max': 10, 'palette':palette}, 'classified' )

# accuracy assessment

test = classified.sampleRegions({
  'collection' : validationPoints,
  'properties' : ['class'],
  'scale': 30
})

print(test)

testConfusionMatrix = test.errorMatrix('class', 'classification')

print('Confusion Matrix ', testConfusionMatrix)

print('Test Accuracy', testConfusionMatrix.accuracy())

# Create a Feature with None geometry and the value we want to export.
# Use .array() to convert Confusion Matrix to an Array so it can be
# exported in a CSV file

fc = ee.FeatureCollection([
  ee.Feature(None, {
    'accuracy': testConfusionMatrix.accuracy(),
    'matrix': testConfusionMatrix.array()
  })
  ])
print(fc)
Export.table.toDrive({
  'collection': fc,
  'description': 'Accuracy_Export',
  'folder': 'ee',
  'fileNamePrefix': 'accuracy',
  'fileFormat': 'CSV'
})

## Calculate variable importance
importance = ee.Dictionary(classifier.explain().get('importance'))

# Calculate relative importance
sum = importance.values().reduce(ee.Reducer.sum())

def func_slk(key, val):
  return (ee.Number(val).multiply(100)).divide(sum)

relativeImportance = importance.map(func_slk)

print(relativeImportance)

# Create a FeatureCollection so we can chart it
importanceFc = ee.FeatureCollection([
  ee.Feature(None, relativeImportance)
])

chart = ui.Chart.feature.byProperty({
  'features': importanceFc
}).setOptions({
      'title': 'Feature Importance',
      'vAxis': '{title': 'Importance'},
      'hAxis': '{title': 'Feature'}
  })
print(chart)

# Tune the numberOfTrees parameter.

test = image.sampleRegions({
  'collection' : validationPoints,
  'properties' : ['class'],
  'scale': 30
})

print(test)
numTreesList = ee.List.sequence(50, 1000, 50)

def func_fbf(numTrees):
  classifier = ee.Classifier.smileRandomForest(numTrees) \
      .train({
        'features': training,
        'classProperty': 'class',
        'inputProperties': image.bandNames()
      })

  # Here we are classifying a table instead of an image
  # Classifiers work on both images and tables
  return test \
    .classify(classifier) \
    .errorMatrix('class', 'classification') \
    .accuracy()

accuracies = numTreesList.map(func_fbf)

print(accuracies)

chart = ui.Chart.array.values({
  'array': ee.Array(accuracies),
  'axis': 0,
  'xLabels': numTreesList
  }).setOptions({
      'title': 'Hyperparameter Tuning for the numberOfTrees Parameters',
      'vAxis': '{title': 'Validation Accuracy'},
      'hAxis': '{title': 'Number of Tress', 'gridlines': '{count': 15}}
  })
print(chart)

# Tuning Multiple Parameters
# numTrees and bagFraction
numTreesList = ee.List.sequence(50,1000, 50)
bagFractionList = ee.List.sequence(0.1, 0.9, 0.1)

def func_ogy(numTrees):
  return bagFractionList.map(function(bagFraction) {
     classifier = ee.Classifier.smileRandomForest({
       'numberOfTrees': numTrees,
       'bagFraction': bagFraction
     }) \
      .train({
        'features': training,
        'classProperty': 'class',
        'inputProperties': image.bandNames()
      })

    # Here we are classifying a table instead of an image
    # Classifiers work on both images and tables
    accuracy = test \
      .classify(classifier) \
      .errorMatrix('class', 'classification') \
      .accuracy()
    return ee.Feature(None, {'accuracy': accuracy,
      'numberOfTrees': numTrees,
      'bagFraction': bagFraction})
  })

accuracies = numTreesList.map(func_ogy
).flatten()

).flatten()
resultFc = ee.FeatureCollection(accuracies)

# Export the result as CSV
Export.table.toDrive({
  'collection': resultFc,
  'description': 'Multiple_Parameter_Tuning_Results',
  'folder': 'ee',
  'fileNamePrefix': 'numtrees_bagfraction',
  'fileFormat': 'CSV'})

MapMap


In [15]:
import ee
import geemap

Map = geemap.Map()

delhi = ee.FeatureCollection("FAO/GAUL/2015/level2").filter(ee.Filter.eq('ADM2_NAME','Delhi')).geometry()

Map.addLayer(delhi)

s2 = ee.ImageCollection("COPERNICUS/S2_SR")

Map.centerObject(delhi)
rgbVis = {
  'min': 0.0,
  'max': 3000,
  'bands': ['B4', 'B3', 'B2'],
}

filtered = s2 \
  .filter(ee.Filter.date('2019-03-01', '2019-05-21')) \
  .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 30)) \
  .filter(ee.Filter.bounds(delhi)) \
  .select('B.*')

before = filtered.median().clip(delhi)
# Display the input composite.
Map.addLayer(before, rgbVis, 'before')

def addIndices(image):
  ndvi = image.normalizedDifference(['B8', 'B4']).rename(['ndvi'])
  ndbi = image.normalizedDifference(['B11', 'B8']).rename(['ndbi'])
  mndwi = image.normalizedDifference(['B3', 'B11']).rename(['mndwi'])
  ndwi = image.normalizedDifference(['B3', 'B8']).rename('ndwi')
  return image.addBands(ndvi).addBands(ndbi).addBands(mndwi).addBands(ndwi)

before = addIndices(before)

data = urban.merge(non_urban)

# Add a random column and split the GCPs into training and validation set
data = data.randomColumn()

# This being a simpler classification, we take 60% points
# for validation. Normal recommended ratio is
# 70% training, 30% validation
training = data.filter(ee.Filter.lt('random', 0.7))
validation = data.filter(ee.Filter.gte('random', 0.7))

# Overlay the point on the image to get training data.
training = before.sampleRegions({
  'collection': training,
  'properties': ['landcover'],
   'tileScale': 16,
  'scale': 10
})

# Train a classifier.
classifier = ee.Classifier.smileRandomForest({
       'numberOfTrees': 50,
     } ).train({
  'features': training,
  'classProperty': 'landcover',
  'inputProperties': before.bandNames()
})

# # Classify the image.
classified = before.classify(classifier)
Map.addLayer(classified,
  {'min': 0, 'max': 3, 'palette': ['gray', 'brown', 'blue', 'green']}, 'before_classified')

print(classified)

classfied = classified.clip(delhi)

# # # # exporting images to asset

# Export.image.toAsset({
#   image: classified,
#   description: 'delhi19_classfiedtest',
#   assetId: 'ee-najah',
#   region: delhi,
#   scale: 100,
#   maxPixels: 1e10
# })

# Export.image.toDrive({
#   image: classified,
#   description: 'delhi_s2_30',
#   scale: 30,
#   region: delhi,
#   maxPixels: 1e10
# })

#**************************************************************************
# Accuracy Assessment
#**************************************************************************

# Use classification map to assess accuracy using the validation fraction
# of the overall training set created above.

test = classified.sampleRegions({
  'collection': validation,
  'properties': ['landcover'],
  'tileScale': 16,
  'scale': 10,
})

testConfusionMatrix = test.errorMatrix('landcover', 'classification')
# Printing of confusion matrix may time out. Alternatively, you can export it as CSV
print('Confusion Matrix', testConfusionMatrix)
print('Test Accuracy', testConfusionMatrix.accuracy())

Export.image.toDrive({
  'image': classified,
  'description': 'delhi_s2_100_clipped',
  'scale': 10,
  'region': delhi,
  'maxPixels': 1e10
})
Map


NameError: name 'urban' is not defined

In [23]:
js_snippet = """
var dataset = ee.ImageCollection('LANDSAT/LT05/C02/T1_L2')
    .filterDate('2000-06-01', '2000-07-01');

// Applies scaling factors.
function applyScaleFactors(image) {
  var opticalBands = image.select('SR_B.').multiply(0.0000275).add(-0.2);
  var thermalBand = image.select('ST_B6').multiply(0.00341802).add(149.0);
  return image.addBands(opticalBands, null, true)
              .addBands(thermalBand, null, true);
}

dataset = dataset.map(applyScaleFactors);

var visualization = {
  bands: ['SR_B3', 'SR_B2', 'SR_B1'],
  min: 0.0,
  max: 0.3,
};

Map.setCenter(-114.2579, 38.9275, 8);

Map.addLayer(dataset, visualization, 'True Color (321)');
  """

In [24]:
geemap.js_snippet_to_py(
    js_snippet, add_new_cell=True, import_ee=True, import_geemap=True, show_map=True
)

In [ ]:
import ee
import geemap

Map = geemap.Map()

dataset = ee.ImageCollection('LANDSAT/LT05/C02/T1_L2') \
    .filterDate('2000-06-01', '2000-07-01')

# Applies scaling factors.
def applyScaleFactors(image):
  opticalBands = image.select('SR_B.').multiply(0.0000275).add(-0.2)
  thermalBand = image.select('ST_B6').multiply(0.00341802).add(149.0)
  return image.addBands(opticalBands, None, True) \
              .addBands(thermalBand, None, True)

dataset = dataset.map(applyScaleFactors)

visualization = {
  'bands': ['SR_B3', 'SR_B2', 'SR_B1'],
  'min': 0.0,
  'max': 0.3,
}

Map.setCenter(-114.2579, 38.9275, 8)

Map.addLayer(dataset, visualization, 'True Color (321)')
Map


In [ ]:
import ee
import geemap

Map = geemap.Map()

visualization = {
  'bands': ['SR_B3', 'SR_B2', 'SR_B1'],
  'min': 0.0,
  'max': 0.3,
}


Map.addLayer(dataset, visualization, 'True Color (321)')
Map


In [ ]:
import ee
import geemap

Map = geemap.Map()

sr2011 = ee.ImageCollection('LANDSAT/LT5_SR') \
  .filterDate('2011-06-20', '2011-08-31') \
  .filter(ee.Filter.eq('WRS_PATH', 59)) \
  .filter(ee.Filter.eq('WRS_ROW', 18))
Map


In [ ]:
ee.Image('LANDSAT/LC08/C01/T1_TOA/LC08_044034_20140318')